# System Design: Real-Time Fraud Detection System

## 1. Requirements Analysis
**Objective**: Detect fraudulent transactions in real-time (< 200ms latency).

**Functional Requirements**:
- Ingest transaction stream.
- Extract features (user history, location, amount).
- Inference via ML model.
- Block/Allow decision.

**Non-Functional Requirements**:
- High Availability (99.99%).
- Low Latency (< 200ms p99).
- Scalability (10k TPS).

## 2. High-Level Architecture

### Data Flow
1. **Transaction Gateway**: Receives request.
2. **Feature Store**: Retrieves pre-computed features (e.g., avg spend last 24h).
3. **Model Service**: Scikit-learn/XGBoost model.
4. **Decision Engine**: Applies business rules + model score.

In [ ]:
import numpy as np
import time
# Use the scratch implementation from src.ml.classical
from src.ml.classical import LogisticRegressionScratch as LogisticRegression
from src.production.monitoring import DriftDetector

# --- Component 1: Feature Store Simulation ---
class FeatureStore:
    def __init__(self):
        self.data = {}
    
    def get_features(self, user_id):
        # Simulate DB lookup latency
        time.sleep(0.005) 
        # Return [avg_spend, transaction_count_24h, risk_score]
        return np.random.rand(3)

# --- Component 2: Model Service ---
class FraudModel:
    def __init__(self):
        self.model = LogisticRegression(learning_rate=0.01)
        # Mock training data
        X_train = np.random.rand(100, 3)
        y_train = np.random.randint(0, 2, 100)
        self.model.fit(X_train, y_train)
        
        # Initialize Monitor with reference data
        self.monitor = DriftDetector()
        self.monitor.set_reference(X_train)
        
    def predict_proba(self, features):
        return self.model.predict_proba(features.reshape(1, -1))

# --- Orchestration ---
feature_store = FeatureStore()
fraud_service = FraudModel()

def process_transaction(user_id, amount):
    start = time.time()
    
    # 1. Feature Engineering
    user_features = feature_store.get_features(user_id)
    
    # 2. Inference
    risk_prob = fraud_service.predict_proba(user_features)
    
    # 3. Monitoring (Drift Detection)
    # In production, this would be async/batch
    drift_results = fraud_service.monitor.detect_drift(user_features.reshape(1, -1))
    
    latency = (time.time() - start) * 1000
    return {
        "user_id": user_id,
        "fraud_probability": risk_prob[0][1] if risk_prob.ndim > 1 else risk_prob,
        "status": "BLOCK" if risk_prob[0][1] > 0.5 else "ALLOW",
        "drift_detected": any(r.drift_detected for r in drift_results),
        "latency_ms": latency
    }

# Simulation
print("Processing transactions...")
for i in range(5):
    result = process_transaction(user_id=101+i, amount=50.0)
    print(f"Transaction {i+1}: {result}")